In [72]:
import whoosh.searching
import whoosh.index as index
from whoosh.qparser import QueryParser

indexdir = "pubmed_index"

ix = index.open_dir(indexdir)

def run_squery(text):
    qp = QueryParser("title", schema=ix.schema)
    q = qp.parse(text)

    with ix.searcher() as s:
        results = s.search(q)
        for hit in results:
            print(f'pmid: {hit["pmid"]}')
            print(f'title: {hit["title"]}')
            print(f'abstract: {hit["abstract"]}')


In [73]:
run_squery("title:penis AND abstract:accident")


pmid: 507742
title: rupture of the penis .
abstract: traumatic rupture of the penis is a rare accident. one or both corpora cavernosa may be affected. approximately one third of the cases are associated with rupture of the urethra. in view of previous studies, and the present case reports both of which occurred during the same night, the authors propose primary surgical repair of the corpora cavernosa .


In [ ]:
from whoosh.qparser import MultifieldParser

def run_mquery(text):
    qp = MultifieldParser(["title", "abstract"], schema=ix.schema)
    q = qp.parse(text)
    with ix.searcher() as s:
        results = s.search(q)
        for hit in results:
            print(f'pmid: {hit["pmid"]}')
            print(f'title: {hit["title"]}')
            print(f'abstract: {hit["abstract"]}')

In [70]:
run_mquery("penis accident")


pmid: 507742
title: rupture of the penis .
abstract: traumatic rupture of the penis is a rare accident. one or both corpora cavernosa may be affected. approximately one third of the cases are associated with rupture of the urethra. in view of previous studies, and the present case reports both of which occurred during the same night, the authors propose primary surgical repair of the corpora cavernosa .
pmid: 600738
title: traumatic rupture of the corpus cavernosum. three cases (author's transl) .
abstract: "fracture of the penis" by damage to the erectile structures is rare. this accident usually occurs when the erect penis is subjected to trauma, usually during intercourse. the diagnosis is easy in the presence of a haematoma spreading from the fraenulum to the scrotum, with sometimes an obvious breach in the albuginosa of one corpus cavernosum. in the three cases reported here, immediate surgery was possible in two. early operation, with repair of the ruptured albuginosa, offers the

In [ ]:
run_squery("title:penis OR abstract:penis AND title:accident OR abstract:accident")

pmid: 520038
title: keratoacanthoma of the glans penis .
abstract: a patient with a solitary keratoacanthoma of the glans penis is presented. since this area is devoid of hair, this keratoacanthoma could not have arisen from a hair follicle as has been suggested as the cause of keratoacanthoma. keratoacanthoma should be included in the differential diagnosis of rapidly growing keratotic tumors of the glans penis. correct diagnosis of this tumor may save patients from needless mutilating surgery of the penis .
pmid: 318755
title: lymphedema of the penis .
abstract: lymphangiectomy with primary closure or skin grafting appears to be the treatment of choice for lymphedema of the penis. the method offers good cosmetic and functional results .
pmid: 22928
title: polyarteritis nodosa of the penis .
abstract: the course of polyarteritis nodosa in a patient showing an isolated nodule localized to the glans penis is described. the underlying disease was not recognised and the patient was wrongl